In [2]:
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import operator

In [14]:
def loadDataSet():
    
    postingList = [['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'], 
                   ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                   ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                   ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                   ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                   ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]#[0,0,1,1,1......]
    classVec = [0, 1, 0, 1, 0, 1] 
    return postingList,classVec

In [11]:
def createVocabList(dataSet):
    #怎么创建单词集合
    #利用集合创建唯一单词列表
    vocabList=set([])
    for line in dataSet:
        #vocabList=vocabList|line
        vocabList=vocabList|set(line)
    return list(vocabList)###################list()不能拉下

In [16]:
def setOfWord2Vec(vocabList,dataLine):
    m=len(vocabList)
    #returnVec=np.zeros((m,1))
    returnVec=[0]*m  #忘了
    for word in dataLine:
        if word in vocabList:### 如何计数 # 创建一个和 唯一词汇表等长的向量，并将其元素都设置为0 
            #如何让对应位置加1 取出单词在vocablist 索引 就是在returnVec 的索引 另其值为1
            #returnVec.index(vocalList.index(word))=1
            returnVec[vocabList.index(word)]=1
        else:
            print('the word %s is not in vocablist ' %word)
    return returnVec

In [119]:
def trainB0(dataMat,labels):
    #1.计算侮辱性类别在总类别中出现的概率 侮辱性类别的个数sum(trainCategory) 0+1+1+0
    numVocab=len(dataMat[0])
    #pAb=sum(labels)/float(len(dataMat))           # float()
    pAb=np.sum(labels)/float(numVocab) 
    #2.定义各类别下 单词出现次数向量 长度是总单词数 定义各类别下单词总数
    
    #p0Num=[1]*numVocab
    p0Num=np.ones(numVocab)
    p1Num=np.ones(numVocab)
    print('type of dataMat',type(dataMat))
    print('type of ponum',type(p0Num))
    p1Denom=2.0
    p0Denom=2.0
    
    #for i in dataMat:
    for i in range(len(dataMat)):
        if labels[i]==1:
            p1Num+=dataMat[i]
            #p1Denom+=sum(dataMat[i])
            p1Denom+=np.sum(dataMat[i])
        else:
            p0Num+=dataMat[i]
            p0Denom+=np.sum(dataMat[i])##################p1Denom+=np.sum(dataMat[i]) 复制成p1
    p1V=np.log(p1Num/p1Denom)              #不能忘log()
    p0V=np.log(p0Num/p0Denom)################np.log() 可以避免下溢出（多个很小数相乘为0），和浮点数四舍五入导致的错误
    print('type of p0v',type(p0V))
    print('p1V',p1V,'\n p0V',p0V,'\n pAb',pAb)
    return p0V,p1V,pAb

In [132]:
def classify(test,p0V,p1V,pAb):
    # 测试数据每个词与其对应的概率相关联起来 相加 比较大小  1.单词在词汇表中的条件下，文件是good 类别的概率 也可以理解为
    print('classify test type',type(test),'p0V test',type(p0V))
    p1=sum(test*p1V)+np.log(pAb)                           ######np.log 不要忘了
    p0=sum(test*p0V)+np.log(1.0-pAb)
    print('p1',p1,'----p0',p0)
    if p1>p0:
        return 1
    else:
        return 0

In [40]:
def test_naive_bayes():
    #1. 加载数据集
    dataList,dataLabels=loadDataSet()
    #2 创建不重复单词集合
    vocabList=createVocabList(dataList)
    # 3 计算单词是否出现 创建每行数据各单词出现次数 矩阵
    trainMat=[]
    for line in dataList:
        trainMat.append(setOfWord2Vec(vocabList,line))
    #4 训练数据得出条件概率
    p0V,p1V,pAb=trainB0(np.array(trainMat),np.array(dataLabels))#构造函数 算出 p(c=1)类别的概率 1类别下各单词出现概率向量 0类别下 各单词出现的概率向量
    #有了条件概率 测试数据
    #5 测试数据 统计测试数据各单词出现的次数向量 构造分类函数
    test=['love', 'my', 'dalmation']
    testDoc=np.array(setOfWord2Vec(vocabList,test))
    # 分类函数 传入条件概率 类别概率
    label=classify(testDoc,p0V,p1V,pAb)
    print('the test result is: {}'.format(label))    
    test_two = ['stupid', 'garbage']
    test_two_doc = np.array(setOfWord2Vec(vocabList, test_two))
    print('the second test  result is: {}'.format(classify(test_two_doc,p0V,p1V,pAb)))       
        

In [120]:
if __name__=='__main__':
    test_naive_bayes()

type of dataMat <class 'numpy.ndarray'>
type of ponum <class 'numpy.ndarray'>
type of p0v <class 'numpy.ndarray'>
p1V [-3.04452244 -3.04452244 -3.04452244 -3.04452244 -2.35137526 -3.04452244
 -3.04452244 -3.04452244 -3.04452244 -3.04452244 -3.04452244 -3.04452244
 -2.35137526 -3.04452244 -1.94591015 -3.04452244 -2.35137526 -3.04452244
 -2.35137526 -1.65822808 -2.35137526 -2.35137526 -2.35137526 -2.35137526
 -3.04452244 -2.35137526 -2.35137526 -1.94591015 -2.35137526 -3.04452244
 -2.35137526 -3.04452244] 
 p0V [-2.56494936 -2.56494936 -2.56494936 -2.56494936 -3.25809654 -2.56494936
 -2.56494936 -2.56494936 -2.56494936 -2.56494936 -2.56494936 -2.56494936
 -3.25809654 -2.56494936 -3.25809654 -1.87180218 -3.25809654 -2.56494936
 -3.25809654 -3.25809654 -2.56494936 -3.25809654 -3.25809654 -2.56494936
 -2.56494936 -3.25809654 -3.25809654 -2.56494936 -3.25809654 -2.56494936
 -2.15948425 -2.56494936] 
 pAb 0.09375
p1 -11.500690927301886 ----p0 -7.100140964637918
the test result is: 0
p1 -6.376

In [17]:
dataList,dataLabels=loadDataSet()
vocabList=createVocabList(dataList)
trainMat=[]
for line in dataList:
    print(line)
    trainMat.append(setOfWord2Vec(vocabList,line))
print(trainMat)
#print(vocabList)

['my', 'dog', 'has', 'flea', 'problems', 'help', 'please']
['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid']
['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him']
['stop', 'posting', 'stupid', 'worthless', 'garbage']
['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him']
['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']
[[1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0]]


In [77]:
def text_parse(text):
    '''
    Desc:
        词条切分
    param:
        传入词条
    return:
        单词列表
    '''
    import re
    word_list=re.split(r'\W+',text)
    if len(word_list)==0:
        print(word_list)
    result=[word.lower() for word in word_list if len(word)>2]
    return result

In [95]:
def bag_word2Vec(vocabList,docLine):
    word_count=[0]*len(vocabList)
    for word in docLine:
        if word in vocabList:
            #word_count[word]+=1
            #word_count[docLine.index(word)]+=1
            word_count[vocabList.index(word)]+=1#######唯一词列表中取索引加1
    return word_count

In [195]:
def span_test():
    '''
    Desc:
        用贝叶斯对垃圾邮件分类
    param:
        None
    return:
        nothing
    '''
    doc_list=[]
    class_list=[]
    full_text=[]
    for i in range(1,26):
        try:
            spam_word=text_parse(open('E:/github/MachineLearning/input/4.NaiveBayes/email/spam/{}.txt'.format(i)).read())
        except:
            spam_word=text_parse(open('E:/github/MachineLearning/input/4.NaiveBayes/email/spam/{}.txt'.format(i),encoding='Windows 1252').read())
        doc_list.append(spam_word)
        full_text.extend(spam_word)
        class_list.append(1)
        
        try:
            ham_word=text_parse(open('E:/github/MachineLearning/input/4.NaiveBayes/email/ham/{}.txt'.format(i)).read())
        except:
            ham_word=text_parse(open('E:/github/MachineLearning/input/4.NaiveBayes/email/ham/{}.txt'.format(i),encoding='Windows 1252').read())
        doc_list.append(ham_word)
        full_text.extend(ham_word)
        class_list.append(0)
        
       
        #print(doc_list)
        
#         print('---------------')
#         print(len(full_text))
#         print(full_text)
#         print('---------------')
#         print(class_list)
#         break
    #print('full_text',full_text) #########################参数不要传错了
    vocabList=createVocabList(doc_list)
    #print('vocabList',vocabList)
    #在50个数随机取10个 这10个做训练数据 50个中删除10个剩下的40个做测试
    
    #calc_most_freq(vocabList,full_text)
    trainingSet=range(50)
    #trainingSet=[]
    testSet=[]
#     for i in range(10):## random.uniform(x, y) 随机生成 一 个范围为 x ~ y 的实数
#         testIndex=int(np.random.uniform(0,50))        ############## int() 不要忘了
#         testSet.append(trainingSet[testIndex])
#         del(trainingSet[testIndex])    ####'range' object doesn't support item deletion
    import random 
    #testSet=[num for num in random.sample(50,10)]
    testSet=[int(num) for num in random.sample(range(50),10)]
    trainingSet=list(set(trainingSet)-set(testSet))
    trainMat=[]
    trainClass=[]
    
    errorCount=0.0
    for i in trainingSet:
        trainMat.append(bag_word2Vec(vocabList,doc_list[i]))
        trainClass.append(class_list[i])
    #print('trainMat',trainMat)
    #p0V,p1V,pAb=trainB0(trainMat,trainClass)
    p0V,p1V,pAb=trainB0(np.array(trainMat),np.array(trainClass))
    for i in testSet:
        testLine=bag_word2Vec(vocabList,doc_list[i])
        #testClass.append(class_list[i])
        #print('\n testLine',testLine)
        label=classify(np.array(testLine),p0V,p1V,pAb)
        if label !=class_list[i]:
            errorCount+=1
            print('classify is %s ,the real label is %s'%(label,class_list[i]))
    errorRatio=errorCount/float(len(testSet))
    print('error ratio is',errorRatio)
    #return errorRatio

In [193]:
if __name__=='__main__':
    span_test()

<class 'dict'>
<class 'list'>
top30 [('the', 50), ('and', 46), ('you', 44), ('100', 28), ('for', 23), ('your', 20), ('that', 17), ('have', 15), ('this', 15), ('yourpenis', 14), ('increase', 14), ('from', 13), ('google', 12), ('with', 11), ('are', 11), ('buy', 11), ('can', 11), ('15mg', 10), ('pills', 10), ('online', 10), ('will', 10), ('answer', 10), ('codeine', 9), ('only', 9), ('more', 9), ('safe', 9), ('all', 9), ('inches', 8), ('experience', 8), ('designed', 8)]
vocab {'the': 50, 'and': 46, 'you': 44, '100': 28, 'for': 23, 'your': 20, 'that': 17, 'have': 15, 'this': 15, 'yourpenis': 14, 'increase': 14, 'from': 13, 'google': 12, 'with': 11, 'are': 11, 'buy': 11, 'can': 11, '15mg': 10, 'pills': 10, 'online': 10, 'will': 10, 'answer': 10, 'codeine': 9, 'only': 9, 'more': 9, 'safe': 9, 'all': 9, 'inches': 8, 'experience': 8, 'designed': 8}
type of dataMat <class 'numpy.ndarray'>
type of ponum <class 'numpy.ndarray'>
type of p0v <class 'numpy.ndarray'>
p1V [-6.50279005 -6.50279005 -6.50

In [194]:
def calc_most_freq(vocabList,full_list):
    vocabCount={}
    for word in full_list:
        if word in vocabList:
            #vocabCount[vocabList.index(word)]+=1
            vocabCount[word]=full_list.count(word)
    import collections
    #print('vocabCount',vocabCount)  
    vocab=[]
    #vocab=dict(Counter(full_list).most_common(30))
    top30=sorted(vocabCount.items(),key=operator.itemgetter(1),reverse=True)[0:30]
    #print(type(vocab))
    #print(type(top30))
    print('top30',top30)
    #print('vocab',vocab)
    return top30

In [197]:
def local_word(feed1,feed0):
    '''
    
    
    '''
    doc_list=[]
    class_list=[]
    full_list=[]
    
    #1 找出两个中最小的一个
    minVal=np.min(len(feed1),len(feed0))
    for i in range(min_len):
        # 类别　１
        word_list = text_parse(feed1['entries'][i]['summary'])
        doc_list.append(word_list)
        full_text.extend(word_list)
        class_list.append(1)
        # 类别　０
        word_list = text_parse(feed0['entries'][i]['summary'])
        doc_list.append(word_list)
        full_text.extend(word_list)
        class_list.append(0)
    vocabList=createVocabList(doc_list)
    
    # 去掉高频词
    top30Word=calc_most_freq(vocabList,full_text)
    #print(top30Word)
    for item in top30Word:
        if item[0] in vocabList:
            vocabList.remove(item[0])
    #随机取数训练
    traingSet=range(50)
    testIndex=[int(num) for num in random.uniform(range(50),10)]
    trainIndex=list(set(trainSet)-set(testIndex))
    trainMat=[]
    trainClass=[]
    for i in trainIndex:
        
        trainMat.append(bag_words2vec(vocabList,doc_list[i]))
        trainClass.append(class_list[i])
    p0V,p1V,pAb=trainB0(np.array(trainMat),np.array(trainClass))
    for i in testIndex:
        test=bag_words2vec(vocabList,doc_list[i])
        #label=classify(np.array(test),np.array(class_list[i]))
        label=classify(np.array(test),p0V,p1V,pAb)
        errorCount=0.0
        if label!=class_list[i]:
            errorCount+=1
    #print('error ratio is %s' %(errorCount/len(doc_list)))
    print('error ratio is %s' %(errorCount/len(testIndex)))
    return vocabList,p0V,p1V

In [196]:
def getTopWord():
    import feedparser
    
    ny = feedparser.parse('http://newyork.craigslist.org/stp/index.rss')
    sf = feedparser.parse('http://sfbay.craigslist.org/stp/index.rss')
    vocabList,p0V,p1V=local_word(ny,sf)
    topNy,topSf=[],[]
    for i in range(len(pOv)):
        if p0V[i]>-6.0:topSf.append(vocabList[i],p0V[i])
        if p1V[i]>-6.0:topNy.append(vocabList[i],p1V[i])
    sorted_ny=sorted(topNy,key=lambda pair:pair[1],reverse=True)
    sorted_sf=sorted(topSf,key=lambda pair:pair[1],reverse=True)
    for item in sorted_ny:
        print(item[0])
    for item in sorted_sf:
        print(item[0])